# Migrating from OpenAI to Open LLMs Using TGI's Messages API

[Text Generation Inference (TGI)](https://github.com/huggingface/text-generation-inference) offers a Messages API, making it directly compatible with the OpenAI Chat Completion API, which means that any existing scripts that use OpenAI models (via the OpenAI client library or third-party tools like LangChain or LlamaIndex) can be directly swapped out to use any Open LLM running on a TGI endpoint.

This allows us to quickly test and benefit from the numerous advantages offered by open models, including:
* complete control and transparency over models and data
* no more worrying about rate limits
* the ability to fully customize systems according to our specific needs

## Setups

In [ ]:
!pip install --upgrade -q huggingface_hub langchain langchain-community langchainhub langchain-openai llama-index chromadb bs4 sentence_transformers torch torchvision torchaudio llama-index-llms-openai-like llama-index-embeddings-huggingface

In [ ]:
import os
import getpass

# enter API key
os.environ["HF_TOKEN"] = HF_API_KEY = getpass.getpass()

## Create an Inference Endpoint

To get started, we need to deploy a [`Nous-Hermes-2-Mixtral-8x7B-DPO`](https://huggingface.co/NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO) model, a fine-tuned Mixtral model, to Inference Endpoint using TGI.

Instead of directly using the [existing UI](https://endpoints.huggingface.co/new?vendor=aws&repository=NousResearch%2FNous-Hermes-2-Mixtral-8x7B-DPO&tgi_max_total_tokens=32000&tgi=true&tgi_max_input_length=1024&task=text-generation&instance_size=2xlarge&tgi_max_batch_prefill_tokens=2048&tgi_max_batch_total_tokens=1024000&no_suggested_compute=true&accelerator=gpu&region=us-east-1) to deploy a model, we will use the Hub library by specifying an endpoint name and model repository, along with the task of `text-generation`.

We will use a `protected` type so access to the deployed model will require a valid HuggingFace token. We also need to configure the hardware requirements like vendor, region, accelerator, instance type, and size.

In [ ]:
from huggingface_hub import create_inference_endpoint

endpoint = create_inference_endpoint(
    'nous-hermes-2-mixtral-8x7b-demo',
    repository='NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO',
    framework='pytorch',
    task='text-generation',
    accelerator='gpu',
    vendor='aws',
    region='us-east-1',
    type='protected',
    instance_type='p4de',
    instance_size='2xlarge',
    custom_image={
        'health_route': '/health',
        'env': {
            'MAX_INPUT_LENGTH': '4096',
            'MAX_BATCH_PREFILL_TOKENS': '4096',
            'MAX_TOTAL_TOKENS': '32000',
            'MAX_BATCH_TOTAL_TOKENS': '1024000',
            'MODEL_ID': '/repository'
        },
        'url': "ghcr.io/huggingface/text-generation-inference:sha-1734540",  # must be >= 1.4.0
    }
)

In [ ]:
endpoint.wait()
endpoint.status

It will take a few minutes for the deployment to spin up. We can use the `.wait()` method to block the running thread until the endpoint reaches a final `"running"` state. Once running, we can confirm its status and test it via the UI Playground.

When deploying with `huggingface_hub`, our endpoint will scale-to-zero after 15 minutes of idle time by default to optimize cost during periods of inactivity.

## Query the Inference Endpoint with OpenAI Client Libraries

Since our model is hosted with TGI, it now supports a Messages API meaning we can query it directly using the faimilar OpenAI client libraries.

In [ ]:
from openai import OpenAI

BASE_URL = endpoint.url

# initialize the client but point it to TGI
client = OpenAI(
    base_url=os.path.join(BASE_URL, 'v1/'),
    api_key=HF_API_KEY
)

In [ ]:
chat_completion = client.chat.completions.create(
    model='tgi',
    messages=[
        {'role': 'system', 'content': "You are a helpful assistant."},
        {'role': 'user', 'content': "Why is open-source software important?"}
    ],
    stream=True,
    max_tokens=512
)

# iterate and print stream
for message in chat_completion:
    print(message.choices[0].delta.content, end="")

TGI's Messages API automatically converts the list of messages into the model's required instruction format using its chat template.

## Integrate with LangChain and LlamaIndex

We can use the newly created endpoint with popular RAG frameworks like LangChain and LlamaIndex.

### Use with LangChain

To use it in LangChain, we simply create an instance of `ChatOpenAI` and pass our `<ENDPOINT_URL>` and `<HF_API_KEY>` as follows:

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name='tgi',
    openai_api_key=HF_API_KEY,
    openai_api_base=os.path.join(BASE_URL, 'v1/')
)

In [ ]:
llm.invoke('Why is open-source software important?')

We now can use our Mixtral model in a simple RAG pipeline to answer a question over the contents of a HF blog post.

In [ ]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# Load the blog contents
loader = WebBaseLoader(
    web_paths=('https://huggingface.co/blog/open-source-llms-as-agents',)
)
docs = loader.load()

# Declare an HF embedding model
hf_embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-large-en-v1.5')

# Chunk and index the docuemnt content
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=hf_embeddings
)

# Retrieve the relevant snippets of the blog
retriever = vectorstore.as_retriever()
prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x['context'])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {
        'context': retriever,
        'question': RunnablePassthrough()
    }
).assign(answer=rag_chain_from_docs)

# Generate the answer
rag_chain_with_source.invoke(
    "According to this article which open-source model is the best for an agent behaviour?"
)

### Use with LlamaIndex

Similarly, we can also use a TGI endpoint in LlamaIndex. We will use the `OpenAILike` class, and instantiate it by comfiguring some additional arguments. Note that the context window argument should match the value previously set for `MAX_TOTAL_TOKENS` of our endpoint.

In [ ]:
from llama_index.llms.openai_like import OpenAILike

llm = OpenAILike(
    model='tgi',
    api_key=HF_API_KEY,
    api_base=BASE_URL + '/v1/',
    is_chat_model=True,
    is_local=False,
    is_function_calling_model=False,
    context_window= 4096, # same as `MAX_TOTAL_TOKENS`
)

In [ ]:
llm.complete('Why is open-source software important?')

We can now use it in a similar RAG pipeline. Note that the previous choice of `MAX_INPUT_LENGTH` in our Inference Endpoint will directly influence the number of retrieved chunks (`similarity_top_k`) the model can process.

In [ ]:
from llama_index.core import VectorStoreIndex, download_loader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.query_engine import CitationQueryEngine

SimpleWebPageReader = download_loader('SimpleWebPageReader')

documents = SimpleWebPageReader(html_to_text=True).load_data(
    ['http://huggingface.co/blog/open-source-llms-as-agents']
)

# Load embedding model
embed_model = HuggingFaceEmbedding(model_name='BAAI/bge-large-en-v1.5')

# Pass LLM to pipeline
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=embed_model,
    show_pregress=True
)

# Query the index
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=2,
)

response = query_engine.query("According to this article which open-source model is the best for an agent behaviour?")
response.response

## Ending

After we are done with our endpoint, we can either pause or delete it.

In [ ]:
endpoint.pause()

In [ ]:
# or
endpoint.delete()